In [ ]:
#Import necessary packages
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

from hydrological_models import HBV
from modelcalibration_camelsde import ModelCalibrationCamelsDE as model_calibration
from evaluation_metrics import nse_loss

In [ ]:
# Initialize information
model = 'HBV' 
path_data = '../../data/CAMELS_DE'
path_output = '../results/conceptual_models_CAMELS_DE/'
path_additional_features = "../../data/CAMELS_DE/pet_hargreaves.pickle"
warmup_period = 365
input_variables = ['precipitation_mean', 'pet(mm/day)', 'temperature_mean']
target_variables = ['discharge_spec']

testing_period = ['1999-01-01','2020-12-31']

hydrological_model = HBV()

# colorblind friendly palette for plotting
color_palette = {"observed": "#1f78b4","simulated": "#ff7f00"}

In [ ]:
# Read the calibration results by each method, and select the best case. In other words, select the calibrated
# parameters (for each basin) that gave best results. The SHM_XXX_summary.csv files are generated when running the 
# scripts associated with each calibration method

# Read DREAM calibration
df_dream = pd.read_csv(path_output + hydrological_model.name + '_dream_summary.csv', dtype={'basin_id': str})
df_dream.set_index('basin_id', inplace=True)
basins_id = df_dream.index
parameter_sets= df_dream.iloc[:, 1:-1].reindex(df_dream.index)

Run the model for each basin, using the best calibration parameters

In [ ]:
test_results = {}
NSE_testing =  []

# Loop that goes through each basin
for i, basin in enumerate(basins_id):
     testing_object = model_calibration(model = hydrological_model,
                                        path_data = path_data,
                                        basin_id = basin,
                                        input_variables = input_variables , 
                                        target_variables = target_variables,
                                        time_period = testing_period, 
                                        obj_func = None, 
                                        warmup_period = warmup_period,
                                        path_additional_features=path_additional_features)
                 
     
     # Testing period ------------------------------------------
     q_sim = testing_object.simulation(parameter_sets.loc[basin].values)
     q_obs = testing_object.evaluation()

     # Calculate loss
     evaluation = q_obs[warmup_period:][testing_object.data_split[warmup_period:]]
     simulation = q_sim[warmup_period:][testing_object.data_split[warmup_period:]]

     time_index = testing_object.timeseries['df'].index[warmup_period:][testing_object.data_split[warmup_period:]]
     df_discharge = pd.DataFrame(data={'y_sim': simulation, 'y_obs': evaluation}, index=time_index)
     test_results[basin] = df_discharge
       
     mask_nans = ~np.isnan(evaluation)
     NSE_testing.append(nse_loss(evaluation=evaluation[mask_nans].flatten(),
                                 simulation=simulation[mask_nans].flatten()))


df_NSE = pd.DataFrame(data={'basin_id': basins_id,'NSE': NSE_testing})
df_NSE = df_NSE.set_index('basin_id')
df_NSE.to_csv(path_output+'/'+model+'_NSE.csv', index=True, header=True)

# Save results as a pickle file
with open(path_output+"/test_results.pickle", "wb") as f:
    pickle.dump(test_results, f)

In [ ]:
# Plot simulated and observed discharges
basin_to_analyze = "DE110350"

plt.rcParams["figure.figsize"] = (20, 8)
plt.plot(test_results[basin_to_analyze]["y_obs"], label="observed", color=color_palette["observed"])
plt.plot(test_results[basin_to_analyze]["y_sim"], label="simulated", alpha=0.5, color=color_palette["simulated"])

# Format plot
plt.xlabel("Day", fontsize=12, fontweight="bold")
plt.ylabel("Discharge [mm/d]", fontsize=12, fontweight="bold")
plt.title("Discharge comparison", fontsize=16, fontweight="bold")
plt.tick_params(axis="both", which="major", labelsize=12)
plt.legend(loc="upper right",fontsize=12)
#plt.savefig(save_folder+"/Model_Comparison.png", bbox_inches="tight", pad_inches=0)